In [ ]:
!pip install fathon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import fathon
from fathon import fathonUtils as fu, DCCA

In [ ]:
# Mapping für die Verbindung zwischen Sentiment und Schlusskursen
name_mapping = {
    'Aixtron': 'AIXA',
    'Hugo Boss': 'BOSS',
    'Redcare Pharmacy': 'RDC',
    'Sartorius': 'STR3',
    'Energiekontor': 'EKT'
}
# Laden der Schlusskursen
file_paths = {
  'EKT':'/content/drive/MyDrive/EKT.XETRA_from_2020-01-02_until_2024-03-01.csv',
  'RDC':'/content/drive/MyDrive/RDC.XETRA_from_2020-01-02_until_2024-03-01.csv',
  'STR3':'/content/drive/MyDrive/SRT3.XETRA_from_2020-01-02_until_2024-03-01.csv',
  'AIXA':'/content/drive/MyDrive/AIXA.XETRA_from_2020-01-02_until_2024-03-01.csv',
  'BOSS':'/content/drive/MyDrive/BOSS.XETRA_from_2020-01-02_until_2024-03-01.csv',
}

# Laden der Sentimentdatei
sentiment_path = '/content/drive/MyDrive/sentiment_results.xlsx'
sentiment_data = pd.read_excel(sentiment_path)


In [ ]:
# Redcare Pharmacy und Shop Apotheke vereinheitlichen
sentiment_data['Aktie'] = sentiment_data['Aktie'].replace(['Shop Apotheke'], 'Redcare Pharmacy').replace(['Sartorius Vz.'], 'Sartorius')

In [ ]:
# Umbenennung der Startdatumsspalte und Konvertierung in ein Datumsformat
sentiment_data.rename(columns={'Startdatum': 'Date'}, inplace=True)
sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date'])

In [ ]:
# Häufigkeiten ausgeben
mention_counts = sentiment_data['Aktie'].value_counts()

print(mention_counts)

Aktie
Aixtron             76
Hugo Boss           74
Redcare Pharmacy    54
Sartorius           45
Energiekontor       32
Name: count, dtype: int64


In [ ]:
# Berechnung der Performance für Tabelle 1
annualized_returns = {}

# Durchlauf aller Dateien und Berechnung der Rendite
for stock, path in file_paths.items():
    data = pd.read_csv(path)
    data['Date'] = pd.to_datetime(data['Date'])

    initial_price = data['Close'].iloc[0]
    final_price = data['Close'].iloc[-1]
    total_days = (data['Date'].iloc[-1] - data['Date'].iloc[0]).days
    simple_return = (final_price - initial_price) / initial_price
    annualized_return = (1 + simple_return) ** (365 / total_days) - 1 if total_days > 0 else simple_return

    annualized_returns[stock] = annualized_return

for stock, a_return in annualized_returns.items():
    print(f"{stock} Annualized Return: {a_return:.2%}")

EKT Annualized Return: 34.10%
RDC Annualized Return: 30.10%
STR3 Annualized Return: 15.77%
AIXA Annualized Return: 31.68%
BOSS Annualized Return: 9.49%


In [ ]:
# Gruppiere die Daten nach Ressort
ressort_counts = sentiment_data.groupby(['Ressort', 'Aktie']).size().reset_index(name="Anzahl Texte")

In [ ]:
# Daten nach Datum
filtered_data = sentiment_data[(sentiment_data['Date'] >= "2020-01-01") & (sentiment_data['Date'] <= "2024-12-31")]

# Gruppierung nach Datum, Aktie und Sentiment
sentiment_trends_by_stock = filtered_data.groupby(
    [pd.Grouper(key="Date", freq="ME"), "Aktie", "Predicted Sentiment"]
).size().reset_index(name="Count")

# Farbzuordnung der Sentiments
color_map = {
    "neutral": "blue",    # Blau für neutral
    "negativ": "red",    # Rot für negativ
    "positiv": "green"   # Grün für positiv
}

# Balkendiagramm erstellen
fig = px.bar(
    sentiment_trends_by_stock,
    x="Date",
    y="Count",
    color="Predicted Sentiment",
    facet_row="Aktie",  # Separate Reihen pro Aktie
    title="Sentiment pro Aktie (2020–2024)",
    labels={"Date": "Zeit", "Predicted Sentiment": "Sentiment", "Aktie": "Aktie"},
    color_discrete_map=color_map  # Farbzuordnung anwenden
)

fig.update_yaxes(matches=None, title=None)  # Entfernt den Text 'Aktie='
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[1]))

# Layout-Anpassungen für größere Grafiken
fig.update_layout(
    height=700,
    margin=dict(t=80, b=50, l=50, r=50),
    title=dict(font=dict(size=28))
)

# Plot anzeigen
fig.show()

In [ ]:
# Berechnung der Wortanzahl für jeden Text
sentiment_data["Wortanzahl"] = sentiment_data["Text"].apply(lambda x: len(x.split()))

sentiment_data = sentiment_data[sentiment_data["Ressort"] != "letzte Seite"]

# Berechnung der durchschnittlichen Wortanzahl pro Ressort
avg_words_per_ressort = sentiment_data.groupby("Ressort")["Wortanzahl"].mean().reset_index()

# Visualisierung
fig = px.bar(
    avg_words_per_ressort,
    x="Ressort",
    y="Wortanzahl",
    title="Durchschnittliche Wortanzahl pro Ressort",
    labels={"Ressort": "Ressort", "Wortanzahl": "Durchschnittliche Wortanzahl"},
    height=600
)

fig.update_layout(
    template="plotly_white",
    title=dict(font=dict(size=28)),
    xaxis=dict(
        title=dict(font=dict(size=24)),
        tickfont=dict(size=20)
      )

)

# Anzeige des Plots
fig.show()

In [ ]:
# Sentimens in numerische Zahlen umwandeln
sentiment_mapping = {'positiv': 1, 'neutral': 0, 'negativ': -1}
sentiment_data['Sentiment_Score'] = sentiment_data['Predicted Sentiment'].map(sentiment_mapping)

In [ ]:
stock_data = {}
scaler = StandardScaler()

for stock, path in file_paths.items():
    df = pd.read_csv(path, parse_dates=['Date'])
    # Standardisierung der Aktienkurse und Speicherung für die nächste Anwendung
    df['Standardized_Close'] = scaler.fit_transform(df[['Close']])
    stock_data[stock] = df[['Date', 'Standardized_Close']]

In [ ]:
# Funktion zur Verknüpfung der Sentiment- und Kursdaten nach Datum
def merge_stock_and_sentiment(stock_data, sentiment_data, stock_symbol):
    merged = pd.merge(
        stock_data[stock_symbol],
        sentiment_data[['Date', 'Lagged_Sentiment']],
        on='Date',
        how='inner' # Nach Übereinstimmung verbinden
    )
    # Setzen des Datums als Index für die Zeitreihenanalyse
    merged.set_index('Date', inplace=True)
    return merged

In [ ]:
lag_days = 1 # Anzahl der Tage, um die Sentiment-Daten zu verschieben
window_size = 15 # Fenstergröße zur Berechnung der Korrelation
rolling_correlations = {}

for sentiment_name, stock_symbol in name_mapping.items():
    # Überprüfe, ob der stock_symbol in stock_data existiert
    if stock_symbol not in stock_data:
        print(f"Fehlender Eintrag für {stock_symbol} in stock_data.")
        continue  # Überspringe fehlende Einträge

    # Filtere das Sentiment für die aktuelle Aktie basierend auf name_mapping
    stock_sentiment = sentiment_data[sentiment_data['Aktie'] == sentiment_name].copy()

    # Verschiebung der Sentiments zur Berücksichtung der Lags und Erstellung der Spalte "Lagged_Sentiment" für den späteren Merge
    stock_sentiment['Lagged_Sentiment'] = stock_sentiment['Sentiment_Score'].shift(lag_days)

    # Verknüpfen der Kurs- und verschobenen Sentimentdaten
    merged_data = merge_stock_and_sentiment(stock_data, stock_sentiment, stock_symbol)

    # Berechnung der rollenden Korrelation
    rolling_correlations[stock_symbol] = merged_data['Standardized_Close'].rolling(window=window_size).corr(merged_data['Lagged_Sentiment'])

In [ ]:
# Visualisierung der Korrelation und Preisdaten für jede Aktie
for stock_symbol, correlation_data in rolling_correlations.items():
    # Zeitraum für Korrelationsergebnisse auswählen
    valid_dates = correlation_data.dropna().index
    valid_correlation = correlation_data.dropna()

    # Filterung der Preisdaten für den Zeitraum der verfügbaren Korrelationsergebnisse
    price_data = stock_data[stock_symbol].set_index('Date').loc[valid_dates]['Standardized_Close']

    fig = go.Figure()

    # Hinzufügen der Preisdaten zur linken y-Achse
    fig.add_trace(go.Scatter(
        x=price_data.index,
        y=price_data,
        name=f'{stock_symbol} Schlusskurse',
        yaxis='y1',
        line=dict(color='blue')
    ))

    # Hinzufügen der rollenden Korrelation zur rechten y-Achse
    fig.add_trace(go.Scatter(
        x=valid_dates,
        y=valid_correlation,
        name=f'{stock_symbol} Korrelation',
        yaxis='y2',
        line=dict(color='purple')
    ))

    # Layout-Einstellungen für die duale Achse
    fig.update_layout(
          xaxis_title='Datum',
          yaxis=dict(
              title='Preisdaten',
              side='left',
              color='blue'
          ),
          yaxis2=dict(
              title='Korrelationskoeffizient',
              overlaying='y',
              side='right',
              color='purple'
          ),
          title=f'Rollende Korrelation und Kursbewegungen {stock_symbol}',
          template='plotly_white'
      )

      # Anzeigen des Plots
    fig.show()


In [ ]:
# Visualisierung der Schlusskurse
fig = go.Figure()

for key, data in stock_data.items():
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Standardized_Close'], mode='lines', name=f'{key} Stock Price'))

fig.update_layout(
  title='Schlusskurse über Zeit',
  title_font=dict(size=24),
  xaxis_title='Datum',
  yaxis_title='Aktienkurs',
  legend_title='Aktie',
  template='plotly_white'
)

fig.show()


In [ ]:
sentiment_data['Aktie'] = sentiment_data['Aktie'].map(name_mapping)

# Überprüfe, ob die Zuordnung erfolgreich war
print(sentiment_data['Aktie'].unique())


['AIXA' 'RDC' 'BOSS' 'EKT' 'STR3']


# Lizenz und Nutzung
Dieses Notebook verwendet die [Fathon-Bibliothek](https://github.com/stfbnc/fathon), die unter der GNU General Public License v3.0 (GPL-3.0) lizenziert ist.

Der hier implementierte Teil zur Detrended Cross-Correlation Analysis (DCCA) unterliegt den Bedingungen der GPL v3.0. Weitere Informationen zur Fathon-Bibliothek sind hier zu finden: (https://github.com/stfbnc/fathon)

In [ ]:
dcca_results = {}

# Iteration über die Aktien im Dictionary
for stock_symbol, stock_df in stock_data.items():

    # Sicherstellen, dass die Datumsformate übereinstimmen
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date'])

    # Filtern der Sentiment-Daten für die aktuelle Aktie
    stock_sentiment = sentiment_data[sentiment_data['Aktie'] == stock_symbol]

    # Zusammenführen der Kursdaten mit den Sentiment-Daten
    merged_data = pd.merge(stock_df, stock_sentiment, on='Date', how='left')

    # Fehlende Werte in der Sentiment-Spalte durch 0 ersetzen
    merged_data['Sentiment_Score'] = merged_data['Sentiment_Score'].fillna(0)

    # Prüfen, ob die Zusammenführung Daten liefert
    if merged_data.empty:
        print(f"Keine Daten für Aktie {stock_symbol} nach Zusammenführung.")
        continue

    # Fenstergrößen definieren
    winSizes = fu.linRangeByStep(5, 200, step=5)
    polOrd = 1  # Polynomialer Grad der Entdetrendung

    # Lag definieren
    lag = 0

    # Verschieben der Sentiment-Daten um Lag 10
    shifted_sentiment = merged_data['Sentiment_Score'].shift(lag).fillna(0)

    # Konvertierung in Numpy-Arrays
    sentiment_series = fu.toAggregated(shifted_sentiment.values)
    stock_series = fu.toAggregated(merged_data['Standardized_Close'].values)

    # Initialisierung des DCCA-Objekts
    pydcca = DCCA(sentiment_series, stock_series)

    # Berechnung der DCCA-Korrelation
    n, rho = pydcca.computeRho(winSizes, polOrd=polOrd)

    # Speichern der Ergebnisse
    dcca_results[f"{stock_symbol}_lag_{lag}"] = {"winSizes": n, "rho": rho}

    # Plotten der DCCA-Korrelationen für Lag 10 mit Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=n,
        y=rho,
        mode='lines+markers',
        name=f'{stock_symbol} (Lag {lag})',
        marker=dict(size=8),
        line=dict(width=2)
    ))

    fig.update_layout(
        title=dict(
            text=f'DCCA-Korrelationen für Aktie {stock_symbol} (Lag {lag})',
            font=dict(size=24)
        ),
        xaxis=dict(
            title=dict(
                text='Fenstergröße (n)',
                font=dict(size=18)
            ),
            tickfont=dict(size=14)
        ),
        yaxis=dict(
            title=dict(
                text=r'$\rho_{DCCA}$',
                font=dict(size=18)
            ),
            tickfont=dict(size=14),
            range=[-1, 1]
        ),
        legend=dict(
            font=dict(size=14),
            orientation="h",
            x=0.5,
            xanchor="center"
        ),
        template="plotly_white"
    )

    # Plot anzeigen
    fig.show()


<ipython-input-39-d8070975c0fb>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Ergebnisse ohne lag
results_summary = {}

for stock_symbol, result in dcca_results.items():
    rho = result['rho']
    results_summary[stock_symbol] = {
        "Mean": np.mean(rho), # Durchschnittliche DCCA-Korrelation
        "Std": np.std(rho), # Standardabweichung der DCCA-Korrelationen
        "Min": np.min(rho), # Minimale Korrelation
        "Max": np.max(rho)  # Maximale Korrelation
    }

# Ergebnisse anzeigen
for stock, stats in results_summary.items():
    print(f"{stock}: Mean={stats['Mean']:.4f}, Std={stats['Std']:.4f}, Min={stats['Min']:.4f}, Max={stats['Max']:.4f}")

EKT: Mean=0.1171, Std=0.1769, Min=-0.2070, Max=0.4669
RDC: Mean=0.1635, Std=0.1094, Min=-0.0504, Max=0.4485
STR3: Mean=0.1320, Std=0.1341, Min=-0.1461, Max=0.3833
AIXA: Mean=0.1953, Std=0.1541, Min=-0.1308, Max=0.4357
BOSS: Mean=0.1078, Std=0.1268, Min=-0.0887, Max=0.4289


In [ ]:
# Ergebnisse mit Lags
results_summary = {}

for stock_symbol, result in dcca_results.items():
    rho = result['rho']
    results_summary[stock_symbol] = {
        "Mean": np.mean(rho), # Durchschnittliche DCCA-Korrelation
        "Std": np.std(rho), # Standardabweichung der DCCA-Korrelationen
        "Min": np.min(rho), # Minimale Korrelation
        "Max": np.max(rho)  # Maximale Korrelation
    }

# Ergebnisse anzeigen
for stock, stats in results_summary.items():
    print(f"{stock}: Mean={stats['Mean']:.4f}, Std={stats['Std']:.4f}, Min={stats['Min']:.4f}, Max={stats['Max']:.4f}")

EKT_lag_10: Mean=0.3366, Std=0.1769, Min=0.0451, Max=0.6614
RDC_lag_10: Mean=0.2399, Std=0.1156, Min=0.0155, Max=0.4673
STR3_lag_10: Mean=0.2119, Std=0.1525, Min=-0.1126, Max=0.5054
AIXA_lag_10: Mean=0.2232, Std=0.1537, Min=-0.0771, Max=0.5332
BOSS_lag_10: Mean=0.1833, Std=0.1501, Min=-0.1035, Max=0.4857
